## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt

In [2]:
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Lompoc,US,34.6391,-120.4579,66.02,66,40,19.57,scattered clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,75.16,84,49,9.26,scattered clouds
2,2,Pevek,RU,69.7008,170.3133,9.61,94,85,5.97,overcast clouds
3,3,Sorland,NO,67.6670,12.6934,41.40,77,97,17.78,overcast clouds
4,4,Naze,JP,28.3667,129.4833,72.52,92,100,10.16,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
1,1,Puerto Ayora,EC,-0.7393,-90.3518,75.16,84,49,9.26,scattered clouds
5,5,Atuona,PF,-9.8000,-139.0333,79.05,75,39,12.84,scattered clouds
7,7,Atar,MR,20.5169,-13.0499,78.78,19,37,6.69,scattered clouds
11,11,Vanimo,PG,-2.6741,141.3028,77.72,89,100,4.45,overcast clouds
12,12,Butaritari,KI,3.0707,172.7902,81.99,75,19,19.48,few clouds
16,16,Nishihara,JP,26.1842,127.7558,75.25,94,75,12.66,broken clouds
18,18,Saint-Philippe,RE,-21.3585,55.7679,76.59,83,85,18.12,light rain
20,20,Cockburn Town,TC,21.4612,-71.1419,78.01,72,0,17.52,clear sky
24,24,Hilo,US,19.7297,-155.0900,84.27,83,100,5.75,overcast clouds
25,25,Bathsheba,BB,13.2167,-59.5167,79.41,69,20,14.97,few clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID            209
City               209
Country            209
Lat                209
Lng                209
Max Temp           209
Humidity           209
Cloudiness         209
Wind Speed         209
Current Weather    209
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,75.16,scattered clouds,-0.7393,-90.3518,
5,Atuona,PF,79.05,scattered clouds,-9.8000,-139.0333,
7,Atar,MR,78.78,scattered clouds,20.5169,-13.0499,
11,Vanimo,PG,77.72,overcast clouds,-2.6741,141.3028,
12,Butaritari,KI,81.99,few clouds,3.0707,172.7902,
16,Nishihara,JP,75.25,broken clouds,26.1842,127.7558,
18,Saint-Philippe,RE,76.59,light rain,-21.3585,55.7679,
20,Cockburn Town,TC,78.01,clear sky,21.4612,-71.1419,
24,Hilo,US,84.27,overcast clouds,19.7297,-155.0900,
25,Bathsheba,BB,79.41,few clouds,13.2167,-59.5167,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")            

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,75.16,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
5,Atuona,PF,79.05,scattered clouds,-9.8000,-139.0333,Villa Enata
7,Atar,MR,78.78,scattered clouds,20.5169,-13.0499,Oumou Elghoura
11,Vanimo,PG,77.72,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
12,Butaritari,KI,81.99,few clouds,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
692,Maceio,BR,80.04,broken clouds,-9.6658,-35.7353,Hotel Ponta Verde Maceió
700,Buchanan,LR,79.47,overcast clouds,5.8808,-10.0467,Kamaneahn Hotel Inc
702,Hajiganj,BD,77.92,clear sky,23.2511,90.8592,Hotel Kohinoor
707,Vieux-Habitants,GP,78.37,overcast clouds,16.0589,-61.7659,Gîtes Dardanelle


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]

max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)

fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))